# 🎉 [Day 24](https://adventofcode.com/2019/day/24)

In [21]:
import numpy as np


def parse_bugs(inputs):
    return np.array([[int(c == '#') for c in line]
                     for line in inputs])


def convolve2d(matrix, kernel): ## From day17
    """Basic 2d convolutions"""
    lkw = int(np.ceil((kernel.shape[0] - 1) / 2))
    lkh = int(np.ceil((kernel.shape[1] - 1) / 2))
    conv = np.zeros_like(matrix)
    # Pad 
    padded_matrix = np.pad(matrix, 
                           ((lkw, kernel.shape[0] - lkw),
                            (lkh, kernel.shape[1] - lkh)),
                            'constant')
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            conv[i, j] = np.sum(kernel * padded_matrix[i:i+kernel.shape[0], 
                                                       j:j+kernel.shape[1]])
    return conv


kernel = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
def game_of_life(bugs):
    """One step of the game of life"""
    global kernel
    counts = convolve2d(bugs, kernel)
    bugs = bugs * (counts == 1) + (1 - bugs) * ((counts == 1) + (counts == 2))
    return bugs


biodiversity_map = np.reshape([2**i for i in range(25)], [5, 5])
def biodiversity_index(bugs):
    global biodiversity_map
    return np.sum(bugs * biodiversity_map)

# Unfortunately biodiversity_index is not layout-unique
# so instead we define unique_index to quickly check for equality
def unique_repr(bugs):
    return ''.join(str(x) for line in bugs for x in line)


def find_cycle(bugs):
    seen = {}
    seen[unique_repr(bugs)] = True
    while 1:
        bugs = game_of_life(bugs)
        x = unique_repr(bugs)
        if x in seen:
            return biodiversity_index(bugs)
        seen[x] = True

In [25]:
with open('inputs/day24.txt', 'r') as f:
    inputs = f.read().splitlines()
    
print("The biodiversity index of the first redundant layout is", find_cycle(parse_bugs(inputs)))

The biodiversity index of the first redundant layout is 1113073
